# अध्याय 7 एजेंट

 - [1. ओपनएआई एपीआई कुंजी सेट करें](#1. ओपनएआई-एपीआई-कुंजी सेट करें)
 - [2. लैंगचेन बिल्ट-इन टूल्स] (#2. लैंगचेन बिल्ट-इन टूल्स)
     - [2.1 एलएलएम-गणित और विकिपीडिया टूल का उपयोग करें](#2.1-एलएम-गणित और विकिपीडिया टूल का उपयोग करें)
     - [2.2 PythonREPLTool टूल का उपयोग करें](#2.2- PythonREPLTool टूल का उपयोग करें)
 - [3. अपने स्वयं के उपकरण परिभाषित करें और उन्हें एजेंट में उपयोग करें] (#3. - अपने स्वयं के उपकरण परिभाषित करें और उन्हें एजेंट में उपयोग करें)
     - [3.1 कस्टम टाइम टूल बनाएं और उपयोग करें](#3.1-कस्टम टाइम टूल बनाएं और उपयोग करें)

## 1. OpenAI API कुंजी सेट करें

एपीआई कुंजी प्राप्त करने के लिए [ओपनएआई खाते] (https://platform.openai.com/account/api-keys) में लॉग इन करें, और फिर इसे एक पर्यावरण चर के रूप में सेट करें।

- यदि आप इसे वैश्विक पर्यावरण चर के रूप में सेट करना चाहते हैं, तो आप [ज़िहु लेख] (https://zhuanlan.zhihu.com/p/627665725) का संदर्भ ले सकते हैं।
- यदि आप इसे स्थानीय/प्रोजेक्ट पर्यावरण चर के रूप में सेट करना चाहते हैं, तो इस फ़ाइल निर्देशिका में एक `.env` फ़ाइल बनाएं, फ़ाइल खोलें और निम्नलिखित सामग्री दर्ज करें।

    <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”>
    OPENAI_API_KEY='आपका_api_key'
    </p>
  
  "your_api_key" को अपनी स्वयं की API कुंजी से बदलें

In [1]:
# आवश्यक पैकेज Python-dotenv और openai डाउनलोड करें
# यदि आपको इंस्टॉलेशन प्रक्रिया लॉग देखने की आवश्यकता है, तो आप -q हटा सकते हैं
!pip install -q python-dotenv
!pip install -q openai

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# स्थानीय/परियोजना पर्यावरण चर पढ़ें।

# find_dotenv() .env फ़ाइल का पथ ढूंढता है और उसका पता लगाता है
#load_dotenv() .env फ़ाइल को पढ़ता है और उसमें मौजूद पर्यावरण चर को वर्तमान चल रहे वातावरण में लोड करता है
# यदि आप एक वैश्विक पर्यावरण चर सेट करते हैं, तो कोड की इस पंक्ति का कोई प्रभाव नहीं पड़ता है।
_ = load_dotenv(find_dotenv())

# पर्यावरण चर OPENAI_API_KEY प्राप्त करें
openai.api_key = os.environ['OPENAI_API_KEY']  

## 2. लैंगचेन अंतर्निर्मित उपकरण

In [2]:
# यदि आपको इंस्टॉलेशन प्रक्रिया लॉग देखने की आवश्यकता है, तो आप -q हटा सकते हैं
# -यू विकिपीडिया के नवीनतम संस्करण को स्थापित करता है इसका कार्य --अपग्रेड के समान है
!pip install -U -q wikipedia
!pip install -q openai

In [3]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

### 2.1 एलएलएम-गणित और विकिपीडिया टूल का उपयोग करना

#### 1️⃣ बड़े भाषा मॉडल को प्रारंभ करें

- डिफ़ॉल्ट कुंजी `openai_api_key` पर्यावरण चर `OPENAI_API_KEY` है। इसलिए निम्नलिखित कोड चलाने से पहले, सुनिश्चित करें कि आपने पर्यावरण चर `OPENAI_API_KEY` सेट कर लिया है। यदि आपके पास अभी तक कोई कुंजी नहीं है, तो कृपया [अपनी एपीआई कुंजी प्राप्त करें](https://platform.openai.com/account/api-keys)।
- डिफ़ॉल्ट मॉडल `model_name` `gpt-3.5-turbo` है।
- मॉडल के डिफ़ॉल्ट मापदंडों के बारे में अधिक जानकारी के लिए, कृपया [यहां] (https://github.com/hwchase17/langchin/blob/master/langchin/chat_models/openai.py) देखें।

In [4]:
# उत्पन्न उत्तरों की यादृच्छिकता को कम करने के लिए पैरामीटर तापमान 0.0 पर सेट किया गया है।
llm = ChatOpenAI(temperature=0)

#### 2️⃣ टूलकिट लोड करें
- `एलएलएम-मैथ` टूल गणितीय गणना करने के लिए भाषा मॉडल और कैलकुलेटर को जोड़ता है
- `विकिपीडिया` टूल खोज प्रश्नों के लिए एपीआई के माध्यम से विकिपीडिया से जुड़ता है।

In [5]:
tools = load_tools(
    ["llm-math","wikipedia"], 
    llm=llm #第一步初始化的模型
)

#### 3️⃣ प्रॉक्सी प्रारंभ करें

- `एजेंट`: एजेंट प्रकार। यहां जो प्रयोग किया गया है वह `AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION` है। उनमें से, `CHAT` का अर्थ है कि एजेंट मॉडल संवाद के लिए अनुकूलित एक मॉडल है, और `REACT` का अर्थ है REACT के लिए डिज़ाइन किया गया एक त्वरित टेम्पलेट।
- `handle_parsing_errors`: पार्सिंग त्रुटियों को संभालना है या नहीं। जब कोई पार्सिंग त्रुटि होती है, तो सुधार के लिए त्रुटि जानकारी बड़े मॉडल पर वापस कर दी जाती है।
- `वर्बोज़`: मध्यवर्ती चरणों के परिणामों को आउटपुट करना है या नहीं।

In [6]:
agent= initialize_agent(
    tools, #第二步加载的工具
    llm, #第一步初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)

#### 4️⃣.1️⃣ गणित के प्रश्नों का उत्तर देने के लिए एजेंटों का उपयोग करें

In [7]:
agent("计算300的25%，思考过程请使用中文。") 



> Entering new  chain...
Question: 计算300的25%
Thought: 我可以使用计算器来计算这个百分比
Action:
```
{
  "action": "Calculator",
  "action_input": "300 * 25 / 100"
}
```

Observation: Answer: 75.0
Thought:我现在知道最终答案了
Final Answer: 300的25%是75.0

> Finished chain.


{'input': '计算300的25%，请用中文', 'output': '300的25%是75.0'}

✅ **सारांश**

1. मॉडल इस पर विचार देता है कि आगे क्या करने की आवश्यकता है।
    
   <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> <strong>सोच</strong>: हमें 300 में से 25% की गणना करने की आवश्यकता है, और इस प्रक्रिया में गुणा और भाग की आवश्यकता होती है। </p>

2. मॉडल सोच के आधार पर कार्रवाई करता है (क्रिया)
    <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> <strong>कार्रवाई</strong>: कैलकुलेटर का उपयोग करें और 300*0.25 दर्ज करें</p>
3. मॉडल का अवलोकन किया गया है (अवलोकन)
    <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”><strong>अवलोकन</strong>: उत्तर: 75.0</p>

4. अवलोकनों के आधार पर, मॉडल यह विचार देता है कि आगे क्या करने की आवश्यकता है।
    <p style='font-family:verdana; फ़ॉन्ट-size:12px;रंग:हरा"> <strong>सोच</strong>: हमारे पास हमारे सवालों के जवाब हैं </p>

5. अंतिम उत्तर दें (अंतिम उत्तर)
     <p style='font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा'> <strong>अंतिम उत्तर</strong>: 75.0 </p>
5. अंतिम उत्तर शब्दकोश रूप में दें।

#### 4️⃣.2️⃣ टॉम एम. मिशेल की किताब

In [18]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
agent(question) 



> Entering new  chain...
I need to find out what book Tom M. Mitchell wrote.
Action: Search for Tom M. Mitchell's books.
Action Input: "Tom M. Mitchell books"
Observation: Search for Tom M. Mitchell's books. is not a valid tool, try another one.
Thought:I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

...

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

> Finished chain.


{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

✅ **सारांश**

1. मॉडल इस पर विचार देता है कि आगे क्या करने की आवश्यकता है।
   <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> <strong>सोच रहा हूं</strong>: मुझे खोजने के लिए विकिपीडिया का उपयोग करना चाहिए। </p>

2. मॉडल सोच के आधार पर कार्रवाई करता है (क्रिया)
    <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> <strong>कार्रवाई</strong>: विकिपीडिया का उपयोग करते हुए, टॉम एम. मिशेल दर्ज करें</p>
3. मॉडल का अवलोकन किया गया है (अवलोकन)
    <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”><strong>अवलोकन</strong>: पेज: टॉम एम. मिशेल, पेज: टॉम मिशेल (ऑस्ट्रेलियाई फुटबॉल खिलाड़ी)</p >

4. अवलोकनों के आधार पर, मॉडल यह विचार देता है कि आगे क्या करने की आवश्यकता है।
    <p style='font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा"> <strong>सोच</strong>: टॉम एम. मिशेल द्वारा लिखित पुस्तक मशीन लर्निंग है </p>

5. अंतिम उत्तर दें (अंतिम उत्तर)
     <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> <strong>अंतिम उत्तर</strong>: मशीन लर्निंग </p>
5. अंतिम उत्तर शब्दकोश रूप में दें।


यह ध्यान देने योग्य है कि मॉडल की अनुमान प्रक्रिया हर बार भिन्न हो सकती है, लेकिन अंतिम परिणाम सुसंगत होते हैं।

### 2.2 PythonREPLTool टूल का उपयोग करना

#### 1️⃣ पायथन एजेंट बनाएं

In [8]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool

agent = create_python_agent(
    llm,  #使用前面一节已经加载的大语言模型
    tool=PythonREPLTool(), #使用Python交互式环境工具（REPLTool）
    verbose=True #输出中间步骤
)

#### 2️⃣ ग्राहक नामों को क्रमबद्ध करने के लिए प्रॉक्सी का उपयोग करें

In [14]:
customer_list = ["小明","小黄","小红","小蓝","小橘","小绿",]

In [22]:
!pip install -q pinyin

In [23]:
agent.run(f"""在这些客户名字转换为拼音\
并打印输出列表： {customer_list}\
思考过程请使用中文。""") 



> Entering new  chain...
我需要使用拼音库来将客户名字转换为拼音。我可以使用Python的pinyin库来实现这个功能。
Action: Python_REPL
Action Input: import pinyin
Observation: 
Thought:我现在可以使用pinyin库来将客户名字转换为拼音了。
Action: Python_REPL
Action Input: names = ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']
Observation: 
Thought:我现在可以使用pinyin库将names列表中的客户名字转换为拼音了。
Action: Python_REPL
Action Input: pinyin_names = [pinyin.get(i, format='strip', delimiter='') for i in names]
Observation: 
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

我已经成功将客户名字转换为拼音并存储在pinyin_names列表中了。
Final Answer: pinyin_names = ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaojv', 'xiaolv']

> Finished chain.


"pinyin_names = ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaojv', 'xiaolv']"

#### 3️⃣ डिबग मोड का उपयोग करें

डिबग मोड में फिर से चलाएँ, हम उपरोक्त 6 चरणों को नीचे दी गई विशिष्ट प्रक्रिया में मैप कर सकते हैं।
1. मॉडल इस पर विचार देता है कि आगे क्या करने की आवश्यकता है।
    - <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> [चेन/स्टार्ट] [1:चेन:एजेंट एक्ज़ीक्यूटर] इनपुट के साथ चेन रन में प्रवेश</p>
    - <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> [चेन/स्टार्ट] [1:चेन:एजेंट एक्ज़ीक्यूटर > 2:चेन:एलएलएमचेन] इनपुट के साथ चेन रन में प्रवेश</p>
    - <p style='font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा'> [llm/start] [1:चेन:AgentExecutor > 2:चेन:LLMChain > 3:llm:ChatOpenAI] एलएलएम में प्रवेश इनपुट के साथ चलता है</p>
    - <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> [llm/end] [1:चेन:AgentExecutor > 2:चेन:LLMChain > 3:llm:ChatOpenAI] [12.25s ] बाहर निकलने वाला एलएलएम आउटपुट के साथ चलता है</p>
    - <p style='font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा'>[चेन/एंड] [1:चेन:एजेंट एक्ज़ीक्यूटर > 2:चेन:एलएलएमचेन] [12.25s] एग्जिटिंग चेन आउटपुट के साथ चलती है </p>
2. मॉडल सोच के आधार पर कार्रवाई (एक्शन) करता है क्योंकि उपयोग किए गए उपकरण अलग हैं, कार्रवाई का आउटपुट भी पहले से अलग है।
    - <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;color:green"> [टूल/स्टार्ट] [1:चेन:एजेंट एक्ज़ीक्यूटर > 4:टूल:पायथन आरईपीएल] इनपुट के साथ एंटरिंग टूल रन</p>
    - <p style='font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा'> [टूल/अंत] [1:चेन:एजेंट एक्ज़ीक्यूटर > 4:टूल:पायथन आरईपीएल] [2.223999999999998एमएस] एग्जिट टूल के साथ चलाएं आउटपुट</p>
3. मॉडल का अवलोकन किया गया है (अवलोकन)
    - <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> [चेन/स्टार्ट] [1:चेन:एजेंट एक्ज़ीक्यूटर > 5:चेन:एलएलएमचेन] इनपुट के साथ चेन रन में प्रवेश</p>
4. अवलोकनों के आधार पर, मॉडल यह विचार देता है कि आगे क्या करने की आवश्यकता है।- <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> [llm/start] [1:चेन:AgentExecutor > 5:चेन:LLMChain > 6:llm:ChatOpenAI] एलएलएम रन में प्रवेश करना इनपुट के साथ</p>
    - <p style='font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा'> [llm/end] [1:चेन:AgentExecutor > 5:चेन:LLMChain > 6:llm:ChatOpenAI] [6.94s ] बाहर निकलने वाला एलएलएम आउटपुट के साथ चलता है</p>
    
5. अंतिम उत्तर दें (अंतिम उत्तर)
    - <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> [श्रृंखला/अंत] [1:श्रृंखला:AgentExecutor > 5:श्रृंखला:LLMChain] [6.94s] एग्जिट चेन आउटपुट के साथ चलती है</p>
6. अंतिम उत्तर लौटाएँ.
    - <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> [श्रृंखला/अंत] [1:श्रृंखला:AgentExecutor] [19.20s] निकास श्रृंखला आउटपुट के साथ चलती है</p>

In [25]:
import langchain
langchain.debug=True
agent.run(f"""在这些客户名字转换为拼音\
并打印输出列表： {customer_list}\
思考过程请使用中文。""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "在这些客户名字转换为拼音并打印输出列表： ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']思考过程请使用中文。"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "在这些客户名字转换为拼音并打印输出列表： ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']思考过程请使用中文。",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just retu

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

[llm/end] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] [12.76s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "我现在可以使用pinyin库来将客户名字转换为拼音。\nAction: Python_REPL\nAction Input: names = ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']\npinyin_names = [pinyin.get(i, format='strip', delimiter='') for i in names]\nprint(pinyin_names)",
        "generation_info": null,
        "message": {
          "content": "我现在可以使用pinyin库来将客户名字转换为拼音。\nAction: Python_REPL\nAction Input: names = ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']\npinyin_names = [pinyin.get(i, format='strip', delimiter='') for i in names]\nprint(pinyin_names)",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 379,
      "completion_tokens": 89,
      "total_tokens": 468
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 5:chain:LLMChain] [12.77s] Exiti

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

[llm/end] [1:chain:AgentExecutor > 8:chain:LLMChain > 9:llm:ChatOpenAI] [20.25s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "我现在知道了最终答案。\nFinal Answer: ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaoju', 'xiaolv']",
        "generation_info": null,
        "message": {
          "content": "我现在知道了最终答案。\nFinal Answer: ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaoju', 'xiaolv']",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 500,
      "completion_tokens": 43,
      "total_tokens": 543
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 8:chain:LLMChain] [20.25s] Exiting Chain run with output:
{
  "text": "我现在知道了最终答案。\nFinal Answer: ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaoju', 'xiaolv']"
}
[chain/end] [1:chain:AgentExecutor] [35.93s] Exiting Chain run with output:
{
  "o

## 3. अपने स्वयं के टूल को परिभाषित करें और उन्हें एजेंट में उपयोग करें

In [26]:
# यदि आपको इंस्टॉलेशन प्रक्रिया लॉग देखने की आवश्यकता है, तो आप -q हटा सकते हैं
!pip install -q DateTime

### 3.1 कस्टम टाइम टूल बनाएं और उपयोग करें

In [27]:
# टूल फ़ंक्शन डेकोरेटर आयात करें
from langchain.agents import tool
from datetime import date

#### 1️⃣ कस्टम टूल बनाने के लिए टूल फ़ंक्शन डेकोरेटर का उपयोग करें
टूल फ़ंक्शन डेकोरेटर को किसी भी फ़ंक्शन पर लागू किया जा सकता है, फ़ंक्शन को लॉन्गचेन टूल में परिवर्तित किया जा सकता है, जिससे यह एजेंट द्वारा कॉल करने योग्य टूल बन जाता है।

हमें फ़ंक्शन में एक बहुत विस्तृत दस्तावेज़ीकरण स्ट्रिंग जोड़ने की आवश्यकता है ताकि एजेंट को पता चले कि किस परिस्थिति में और फ़ंक्शन/टूल का उपयोग कैसे करना है।

उदाहरण के लिए, निम्नलिखित फ़ंक्शन `समय` के लिए, हमने एक विस्तृत दस्तावेज़ स्ट्रिंग जोड़ी है

```अजगर
"""
आज की तारीख लौटाता है, आज की तारीख प्राप्त करने से संबंधित किसी भी समस्या के लिए उपयोगी।
इनपुट हमेशा एक खाली स्ट्रिंग होना चाहिए और फ़ंक्शन हमेशा आज की तारीख लौटाएगा।
किसी भी तिथि की गणना इस फ़ंक्शन के बाहर की जानी चाहिए।
"""

```

In [28]:
@tool
def time(text: str) -> str:
    """
    返回今天的日期，用于任何需要知道今天日期的问题。\
    输入应该总是一个空字符串，\
    这个函数将总是返回今天的日期，任何日期计算应该在这个函数之外进行。
    """
    return str(date.today())

#### 2️⃣ प्रॉक्सी प्रारंभ करें

In [36]:
agent= initialize_agent(
    tools=[time], #将刚刚创建的时间工具加入代理
    llm=llm, #初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)

#### 3️⃣ आज की तारीख पूछने के लिए प्रॉक्सी का उपयोग करें
**नोट**: प्रॉक्सी कभी-कभी गलत हो सकती है (यह सुविधा विकास के अधीन है)। यदि कोई त्रुटि आती है, तो उसे पुनः चलाने का प्रयास करें।

In [37]:
agent("今天的日期是？") 



> Entering new  chain...
根据提供的工具，我们可以使用`time`函数来获取今天的日期。

Thought: 使用`time`函数来获取今天的日期。

Action:
```
{
  "action": "time",
  "action_input": ""
}
```


Observation: 2023-07-04
Thought:我现在知道了最终答案。
Final Answer: 今天的日期是2023-07-04。

> Finished chain.


{'input': '今天的日期是？', 'output': '今天的日期是2023-07-04。'}

✅ **सारांश**

1. मॉडल इस पर विचार देता है कि आगे क्या करने की आवश्यकता है।
   <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> <strong>सोच रहा हूं</strong>: आज की तारीख जानने के लिए मुझे टाइम टूल का उपयोग करना होगा</p>

2. मॉडल सोच के आधार पर कार्रवाई (एक्शन) करता है क्योंकि उपयोग किए गए उपकरण अलग हैं, कार्रवाई का आउटपुट भी पहले से अलग है।
    <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> <strong>कार्रवाई</strong>: समय उपकरण का उपयोग करें और एक खाली स्ट्रिंग दर्ज करें</p>
    
   
3. मॉडल का अवलोकन किया गया है (अवलोकन)
    <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”><strong>अवलोकन</strong>: 2023-07-04</p>
    
   
    
4. अवलोकनों के आधार पर, मॉडल इस पर विचार देता है कि आगे क्या करने की आवश्यकता है (हालांकिटी)
    <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> <strong>सोच रहा हूं</strong>: मैंने टाइम टूल का उपयोग करके आज की तारीख सफलतापूर्वक प्राप्त कर ली है</p>

5. अंतिम उत्तर दें (अंतिम उत्तर)
     <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”> <strong>अंतिम उत्तर</strong>: आज की तारीख 2023-07-04 है।</p>
6. अंतिम उत्तर लौटाएँ.